In [1]:
import pandas as pd
import os

In [2]:
local_modules_path = '../src'
os.sys.path.append(local_modules_path)

In [3]:
import api_fetcher

This notebooks was created with the aim of visualizing Tennis demand and create a simple ML model

First, how can we get this data? We are able to pull data from the Amazon's public API via rapidapi. So, we need to search in the product categories, where the Tennis is. It might be likely needed to pull both Women and Men Tenis. Let's take a look at the Women Tenis

In [8]:
amazon_api_client = api_fetcher.ApiClient()
amazon_api_client.http_request(
    method="GET",
    request_url="/products-by-category?category_id=2478868012&page=1&country=US&sort_by=RELEVANCE&product_condition=ALL&is_prime=false&deals_and_discounts=NONE"
)
amazon_api_client.save_response(filename='product_list_us')

In [10]:
import json

with open('../data/api-calls/us_product_list.json', 'r') as file:
    product_list_json = json.load(file)

In [15]:
api_call_response = eval(product_list_json['response'])

In [17]:
api_call_data = api_call_response['data']

In [20]:
pd.DataFrame(api_call_data).sort_values(by="name")

,name,id
12,AWS Courses,courses
1,Alexa Skills,alexa-skills
0,All Departments,aps
2,Amazon Devices,amazon-devices
3,Amazon Explore,live-explorations
4,Amazon Fresh,amazonfresh
5,Amazon Pharmacy,amazon-pharmacy
6,Amazon Warehouse,warehouse-deals
7,Appliances,appliances
8,Apps & Games,mobile-apps


It seems we need to delve depet into to both `fashion-mens` and `fashion-womens`

Let's try out with the path parameter `product-details`

In [47]:
amazon_api_client.http_request(
    method="GET",
    request_url="/product-details?asin=B0BZXSLD5H&country=US"
)

In [48]:
amazon_api_client.save_response(filename="prod_details_us")

In [49]:
with open(
    "../data/api-calls/prod_details_us.json",
    'r'
) as file:
    prod_details_json = json.load(file)

In [50]:
product_data = json.loads(prod_details_json['response'])['data']

In [51]:
product_data

{'asin': 'B0BZXSLD5H',
 'product_title': "Under Armour men's Charged Commit Trainer 4 Sneaker",
 'product_price': '$39.20',
 'product_original_price': None,
 'product_price_max': '$69.99',
 'currency': 'USD',
 'country': 'US',
 'product_byline': 'Visit the Under Armour Store',
 'product_star_rating': '4.6',
 'product_num_ratings': 392,
 'product_url': 'https://www.amazon.com/dp/B0BZXSLD5H',
 'product_photo': 'https://m.media-amazon.com/images/I/51Ih-2zQ++L._AC_SL1200_.jpg',
 'product_num_offers': 1,
 'product_availability': None,
 'is_best_seller': False,
 'is_amazon_choice': False,
 'is_prime': False,
 'climate_pledge_friendly': False,
 'sales_volume': None,
 'about_product': ['Under Armour’s mission is simple: to make you better.',
  'That means if you set a goal, we’ll help you crush it.',
  'We do that with performance innovation and game-changing shoes, shirts, and digital tools that push the limits of what you thought possible.'],
 'product_description': 'Lightweight mesh upper w

In [53]:
prod_details_df = pd.DataFrame(
    data=product_data
)

ValueError: All arrays must be of the same length

In [28]:
prod_details_df

,name,id
0,All Departments,aps
1,Alexa Skills,alexa-skills
2,Amazon Devices,amazon-devices
3,Amazon Explore,live-explorations
4,Amazon Fresh,amazonfresh
5,Amazon Pharmacy,amazon-pharmacy
6,Amazon Warehouse,warehouse-deals
7,Appliances,appliances
8,Apps & Games,mobile-apps
9,"Arts, Crafts & Sewing",arts-crafts


In [26]:
import http.client

conn = http.client.HTTPSConnection("real-time-amazon-data.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "dfa8842b83msha2bc48dbc5792bdp1cbbd0jsn0bde68569041",
    'x-rapidapi-host': "real-time-amazon-data.p.rapidapi.com"
}

conn.request("GET", "/product-details?asin=B07ZPKBL9V&country=US", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"status":"OK","request_id":"550bcf63-d4bb-47cb-a00d-8f9ab242c13c","parameters":{"asin":"B07ZPKBL9V","country":"US"},"data":{"asin":"B07ZPKBL9V","product_title":"Apple iPhone 11, 64GB, (PRODUCT)RED - Fully Unlocked (Renewed)","product_price":"$222.00","product_original_price":null,"currency":"USD","country":"US","product_byline":"Visit the Amazon Renewed Store","product_star_rating":"4.2","product_num_ratings":53023,"product_url":"https://www.amazon.com/dp/B07ZPKBL9V","product_photo":"https://m.media-amazon.com/images/I/514k7uOBMwL._AC_SL1000_.jpg","product_num_offers":32,"product_availability":"In Stock","is_best_seller":false,"is_amazon_choice":false,"is_prime":false,"climate_pledge_friendly":true,"sales_volume":"500+ bought in past month","about_product":["This phone is unlocked and compatible with any carrier of choice on GSM and CDMA networks (e.g. AT&T, T-Mobile, Sprint, Verizon, US Cellular, Cricket, Metro, Tracfone, Mint Mobile, etc.).","Tested for battery health and guaranteed